# 치터 판별 XGBoost 모델 학습

Optical Flow로 추출된 프레임 데이터를 전처리하고, **해당 영상의 플레이어가 치터인지 아닌지 판별**하는 XGBoost 이진 분류 모델을 학습

---

### 주요 동작 흐름

1.  **데이터 로드**:
    * 일반 유저와 치터의 데이터가 담긴 Excel 파일을 각각 로드
2.  **데이터 전처리**:
    * 각 데이터에 치트 여부를 나타내는 라벨을 추가
    * 모델 학습에 사용할 주요 특징을 정의
    * 데이터를 학습용과 검증용으로 분할
3.  **XGBoost 모델 학습**:
    * 클래스 불균형 문제를 완화하기 위해 파라미터를 적용
    * 하이퍼파라미터를 설정하고, 모델을 학습
4.  **모델 성능 평가**:
    * 학습된 모델의 성능을 평가

---

### 핵심 포인트

* **입력 데이터**: 각 프레임 단위로 계산된 에임의 위치 변화량, 속도, 가속도 등을 사용
* **특징 정의**: 프레임 데이터를 전처리하여 모델 학습에 사용할 수 있도록 재정의
* **클래스 불균형 처리**: 데이터셋에 치터보다 일반 유저가 훨씬 많기 때문에, 소수 클래스에 더 높은 가중치를 부여하여 모델이 치터 데이터를 더 중요하게 학습하도록 유도

## 라이브러리 설치

In [ ]:
!pip install xgboost shap pandas scikit-learn openpyxl

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, average_precision_score
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

## 데이터 로드

In [ ]:
# --- 엑셀 로드 ---
df_human = pd.read_excel("/content/drive/MyDrive/Preprocessed_Dataset/aim_analysis_human.xlsx")  # 사람 데이터 엑셀 경로 지정
df_cheat = pd.read_excel("/content/drive/MyDrive/Preprocessed_Dataset/aim_analysis_cheat.xlsx") # 치트 데이터 엑셀 경로 지정

# --- 치트 라벨 추가 ---
# 사람 데이터프레임에 모든 행에 'cheat' 컬럼을 추가하고 값을 0으로 설정
df_human['cheat'] = 0
# 치트 데이터프레임의 모든 행에 'cheat' 컬럼을 추가하고 값을 1로 설정
df_cheat['cheat'] = 1

## 데이터 전처리

In [ ]:
# 두 데이터프레임을 하나로 합침
df = pd.concat([df_human, df_cheat], ignore_index=True)

###
# 영상마다 프레임별로 나뉜 데이터를 묶어야 함
#   프레임 별 피처
#   - 위치 변화량(dx, dy), 속도(velocity), 가속도(acceleration), (jerk), (angle_deg), (angle_change)
#   요약 피처
#   - 평균(mean), 표준 편차(std), 최솟값(min), 최댓값(max), 중앙값(median), 이상값 비율
###

# 피처 및 라벨 분리
feature_cols = ['dx', 'dy', 'velocity', 'acceleration', 'jerk', 'angle_deg', 'angle_change']
X = df[feature_cols]
y = df['cheat'] # 'cheat' 컬럼을 라벨로 사용

# 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Data loaded and labels created:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("\nClass distribution in y_train:")
print(y_train.value_counts())

Data loaded and labels created:
X_train shape: (162974, 7)
y_train shape: (162974,)
X_val shape: (40744, 7)
y_val shape: (40744,)

Class distribution in y_train:
cheat
0    146835
1     16139
Name: count, dtype: int64


In [ ]:
df.shape # 데이터 행과 열
df.columns # 데이터 열 종류
df.info() # 데이터 정보

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203718 entries, 0 to 203717
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   filename      203718 non-null  object 
 1   frame         203718 non-null  int64  
 2   dx            203718 non-null  float64
 3   dy            203718 non-null  float64
 4   velocity      203718 non-null  float64
 5   acceleration  203462 non-null  float64
 6   jerk          203206 non-null  float64
 7   angle_deg     203718 non-null  float64
 8   angle_change  203462 non-null  float64
 9   cheat         203718 non-null  int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 15.5+ MB


## XGBoost 모델 학습

In [ ]:
# --- 모델 학습 ---

###
# 하이퍼 파라미터
#   max_depth
#   min_child_weight
#   gamma
#   leraning_rate
#   n_estimators
#   subsample
#   colsample_bytree
#
# 최적 조합 찾기
#   GridSearchCV
#   RandomizedSearchCV
###

# 클래스 불균형 고려
pos = (y_train == 1).sum()
neg = (y_train == 0).sum()
scale_pos_weight = neg / max(pos, 1)

model = xgb.XGBClassifier(
    n_estimators=400,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    objective="binary:logistic",
    eval_metric="auc",
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

ROC-AUC: 0.785913873752315
PR-AUC : 0.3491153093914882


In [ ]:
# --- 모델을 저장할 경로 지정 ---
model_save_path = "/content/drive/MyDrive/Preprocessed_Dataset/"
model_name = "xgboost_cheater_detection_model.json"
full_model_path = os.path.join(model_save_path, model_name)

# --- 모델 저장 ---
os.makedirs(model_save_path, exist_ok=True)
model.save_model(full_model_path)

print(f"모델이 성공적으로 저장되었습니다.")
print(f"저장 경로: " {full_model_path})

## 모델 성능 평가

In [ ]:
# --- 모델 성능 평가 ---

val_proba = model.predict_proba(X_val)[:, 1]
print("ROC-AUC:", roc_auc_score(y_val, val_proba))
print("PR-AUC :", average_precision_score(y_val, val_proba))

# --- 정확도 ---

# --- 혼동 행렬 ---

# --- 분류 리포트 ---

# --- ROC-AUC 점수 및 곡선 ---